# Week 2: Potential Outcomes

The concept of potential outcomes are foundational to the reasoning system that we're going to use for the course. They allow us to think about, with considerably clarity, the comparisons that we want to make, and **exactly** why these comparisons are *causal*. 

But, despite the somewhat obtuse language that we might use when we're talking about potential outcomes (e.g. "The potential outcome to treatment, when a unit is assigned to treatment.") the math surrounding potential outcomes is actually pretty straightforward. 

# Create Data 
Let's make some toy data that we can use through this demo. 

Suppose that we have 1,000 individuals that we can observe in our study. Furthermore, suppose that for each of these people, there is some *latent*, or unmeasured, response to treatment that they *would* have to treatment if we assigned them to take the treatemnt. 

You might think of this as an unknowable population-level parameter that we're trying to estimate from empirical data. In this worksheet, we're going to make the data so that the causal parameter $\tau$ has a mean value of 2. But, you could imagine that some other treatment has a mean value of 100. Or, for some third treatment it could have a mean value of zero, in which case we would say there there is *no* treatment effect. 

For concreteness, suppose that the treatment is assigning people to drink [coffee](https://www.aldeacoffee.com).

In [1]:
import pandas as pd 
import numpy as np 

In [3]:
NROWS = 1000

d = pd.DataFrame({
    'id'  : np.arange(0,NROWS)
})

Let's build more into our data, like, that some of the people are tall, while others are short; and some are young, while others are old. 


In [4]:
d['height'] = np.random.choice(a=['tall', 'short'], replace=True, size=NROWS)
d['age']    = np.random.choice(a=['young', 'old'] , replace=True, size=NROWS)

In [5]:
d.head()

,id,height,age
0,0,short,young
1,1,short,young
2,2,tall,old
3,3,short,young
4,4,tall,old


In [6]:
d.head()

,id,height,age
0,0,short,young
1,1,short,young
2,2,tall,old
3,3,short,young
4,4,tall,old


In just exactly the same way, we can also think of our units having levels of *potential outcomes to control*. That is, we can suppose that people have some level of the outcome in the case that they do not drink any coffee. For concreteness, suppose that the outcome is people's number of minutes of 241 coding they can accomplish, before they fall asleep. 

There might (or might not) be a relationship between our non-experimentally assigned outcomes. In the example we're working with here, suppose that there is no relationship between height and minutes of coding; but that there is a positive relationship between age and minutes of coding. 

This might be represented in our data in the following way: The mean number of minutes that someone can work is 10, plus, if they're old (ahem... seasoned, disciplined) they are able to work for an additional 5 minutes on avergae. But, also assume that there is some noise in this relationship. 

## Potential Outcomes to Control 

In [7]:
d['y0'] = 10 + (d['age'] == 'old') * 2 \
  + np.random.normal(size=NROWS, loc=0, scale=1)

In [8]:
d.head()

,id,height,age,y0
0,0,short,young,9.555415
1,1,short,young,10.444188
2,2,tall,old,11.479307
3,3,short,young,12.241072
4,4,tall,old,12.997396


Notice that there is no relationship between height and potential outcomes to control; and also notice that the "noise" in the relationship is represented in the draw from the normal distribution with mean 0. 

## Potential Outcomes to Treatment 

We've defined a *treatment effect* as the difference between a unit's potential outcome to treatment and their potential outcome to control. But, right now, we've only got one of those, the `y0`. 

Lets suppose that, since caffiene is a drug, and people are people, that there is some common response to the drug. 

However, because people are also people, not everybody reacts in exactly the same way; some people beome more attentive than others; some less than others. 

In [10]:
d['tau'] = np.random.normal(loc=2, scale=2, size=NROWS)

Now, if know each person's *causal effect*, then I suppose we also know their potential outcomes to **treatment**, right?

In [11]:
d['y1'] = d['y0'] + d['tau']

And so, we can represent this **science table** -- the set of all potentially realizable outcomes. 

In [12]:
d.head()

,id,height,age,y0,tau,y1
0,0,short,young,9.555415,2.838994,12.394410
1,1,short,young,10.444188,-3.475198,6.968990
2,2,tall,old,11.479307,1.439453,12.918760
3,3,short,young,12.241072,2.133776,14.374848
4,4,tall,old,12.997396,3.355633,16.353029


## Questions for Understanding 
1. On average, will people who are older tend to be taller, shorter, or about the same height, and people who are younger? 
2. On average, will people who are older tend to have higher or lower potential outcomes to control? 
3. On average, will people who are older tend to have higher or lower potential outcomes to treatment? 
4. **Most importantly**: If these are *potential outcomes* then, can we empirically observe any of these outcomes? 


In [ ]:
# question 1 

In [ ]:
# question 2 

In [ ]:
# question 3

In [ ]:
# question 4 

# Run Your Experiment 

To this point, we've been working in the *potential outcomes space*. One way that you might think about this is as though these are the measurement that our population of people are going to walk into the expeiment having -- but we haven't measured them yet, so we don't know what they are! That is, suppose we have the whole cohort of students who are enrolled in 241, and they're going to start the coffee drinking experiment in Week 3. In Week 2, they all are either young/old, short/tall, and have innate abilities to focus, *we as the experimenters just don't know them yet!*. 

When we run the experiment, we accomplish several things: 

1. We intervene in their lives to force a particular experience; 
2. As a result of our intervention, we *reveal* either potential outcomes to treatment or control for each subject, and we measure this. 
3. We measure outcomes from our subjects;

Each of these require considerable care; the first two are principle goals of this course. The last is *equally* important; and there are entire courses and disciplines that surround the theory or measurement. We're going to work on this as an *off-label* goal of the course. 

In this experiment, the way that we are interviening in people's lives is to either give them *coffee* or *decaf coffee*. 


In [13]:
d['treat'] = np.random.choice(a=[0,1], replace=True, size=NROWS)

Then, and only then, we're also able to make some of their potential outcomes measurable. 

In [14]:
d['Y'] = d['y0'] * (d['treat'] == 0) + d['y1'] * (d['treat'] == 1)

What data do we have now? 

In [15]:
d.head()

,id,height,age,y0,tau,y1,treat,Y
0,0,short,young,9.555415,2.838994,12.394410,0,9.555415
1,1,short,young,10.444188,-3.475198,6.968990,1,6.968990
2,2,tall,old,11.479307,1.439453,12.918760,1,12.918760
3,3,short,young,12.241072,2.133776,14.374848,0,12.241072
4,4,tall,old,12.997396,3.355633,16.353029,1,16.353029


This, above, is the "science table" that is represented in *Field Experiments* as table 2.1. But, are we ever able to see all of these in the experient that we conduct? 

## Estimating Causal Quantities from All of the Data 

Suppose for a moment that we *could* observe parts of the science table. In particular, suppose that we had access to everybody's potential outcomes to treatment and control, but not their treatment effect. Could we make an estimate about the average treatment effect then? Sure! 


In [22]:
calculated_effects = d['y1'] - d['y0']
calculated_effects.head()

0    2.838994
1   -3.475198
2    1.439453
3    2.133776
4    3.355633
dtype: float64

And so, what is the average of these?

In [64]:
calculated_effects.mean()

2.0864820681426584

In [17]:
mean_tau = d['tau'].mean()
mean_tau

1.890161613393389

In [24]:
np.isclose(mean_tau, calculated_effects.mean())

True

## Why only some of? 

Why are we only able to measure some of people's potential outcomes? We're only able to measure the potential outcomes that are consistent with the treatment that we actually give people. 

And, so, while the *science table* might contain information about each persons potential outcomes to treamtent and control, the table of data that we're ever going to be able to generate has a more restricted set. Call this observable data set `obs`, and it is a subset of all the data that might be out there.

In [16]:
obs = d[['id', 'height', 'age', 'treat', 'Y']]
obs.head()

,id,height,age,treat,Y
0,0,short,young,0,9.555415
1,1,short,young,1,6.968990
2,2,tall,old,1,12.918760
3,3,short,young,0,12.241072
4,4,tall,old,1,16.353029


Where did all the data go?? We no longer have access to people's causal effect ($\tau$), nor do we have access to their potential outcomes to control and treatment. Instead, we're left only with their *realized potential outcomes* that match the condition they were assigned to. 

## Question of Understanding 
1. What happens to these values that are no longer in our dataset? Because we can't measure them, does that mean that they don't exist? Does it mean that they *never* existed? 
2. If we can't see any person's $\tau$, then how should we generate an estimate of this population-level parameter? 



## Question for understanding
1. Is the average height of people who are in treatment more or less than the average height for people who are in control? 
2. Is the average potential outcome to control, `y_0`, of people who are in treatment more or less than the average potential outcome to control for people who are in control? 
3. Is the average potential outcome to treatment, `y_1`, of people who are in treatment more or less than the average potential outcome to treatment for people who are in control?
4. Is the average potential outcome to treatment, `y_1` of *people who are in treamtent* more or less than the average potential outcome to control, `y_0` of *people who are in treatment*.



In [29]:
# question 1


In [30]:
# question 2 


In [31]:
# question 3 


In [32]:
# question 4


# Estimating Causal Quantities from the Observable Data 

Of course, we don't get to see each of these potential outcomes instead, we only get to see either one or the other. Which one do we get to see? Well the potential outcome for the treatment condition thatwe assign the person to!

For someone in control, for example, we can see only their potential outcome to control. 

Is this the same as off the *science table* it sure should be! 

The exact same logic applies to the potential outcomes to treamtent as well: 

# The Big Punchline! 

Because the *observable* set of realised potential outcomes are unbiased estimates of the *unobservable* potential outcomes, we're able to generate an unbiased estimate of the causal effect, using only observable data! 

In general the framework looks like this. 

Given random assignment to treatment and control: 

1. Use observable data from each of the randomly assigned group to produce an unbiased estimate of the mean value of the potential outcome for that group. 
2. Use these two unbiased estimators to produce an unbiased estimate of the difference between potential outcomes to treatment and the potential outcomes to control. 
3. This value, while less *precise* than a case where we could measure *tau* for each individual, is an unbiased and consistent estimate of an otherwise unobservable feature. 
